## Membangun Model Regresi Logistik - Pelajaran 4

![Infografis regresi logistik vs. regresi linear](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Kuis sebelum pelajaran](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Pendahuluan

Dalam pelajaran terakhir tentang Regresi ini, salah satu teknik dasar *klasik* dalam pembelajaran mesin, kita akan mempelajari Regresi Logistik. Anda dapat menggunakan teknik ini untuk menemukan pola guna memprediksi kategori biner. Apakah permen ini cokelat atau bukan? Apakah penyakit ini menular atau tidak? Apakah pelanggan ini akan memilih produk ini atau tidak?

Dalam pelajaran ini, Anda akan mempelajari:

- Teknik-teknik untuk regresi logistik

✅ Perdalam pemahaman Anda tentang cara bekerja dengan jenis regresi ini di [modul pembelajaran ini](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Prasyarat

Setelah bekerja dengan data labu, kita sekarang cukup familiar untuk menyadari bahwa ada satu kategori biner yang dapat kita gunakan: `Color`.

Mari kita bangun model regresi logistik untuk memprediksi, berdasarkan beberapa variabel, *warna apa yang kemungkinan dimiliki oleh sebuah labu* (oranye 🎃 atau putih 👻).

> Mengapa kita membahas klasifikasi biner dalam kelompok pelajaran tentang regresi? Hanya untuk kemudahan linguistik, karena regresi logistik sebenarnya adalah [metode klasifikasi](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), meskipun berbasis linear. Pelajari cara lain untuk mengklasifikasikan data di kelompok pelajaran berikutnya.

Untuk pelajaran ini, kita memerlukan paket-paket berikut:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/) adalah [koleksi paket R](https://www.tidyverse.org/packages) yang dirancang untuk membuat ilmu data lebih cepat, mudah, dan menyenangkan!

- `tidymodels`: Kerangka kerja [tidymodels](https://www.tidymodels.org/) adalah [koleksi paket](https://www.tidymodels.org/packages/) untuk pemodelan dan pembelajaran mesin.

- `janitor`: Paket [janitor](https://github.com/sfirke/janitor) menyediakan alat sederhana untuk memeriksa dan membersihkan data yang kotor.

- `ggbeeswarm`: Paket [ggbeeswarm](https://github.com/eclarke/ggbeeswarm) menyediakan metode untuk membuat plot gaya beeswarm menggunakan ggplot2.

Anda dapat menginstalnya dengan perintah berikut:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Sebagai alternatif, skrip di bawah ini akan memeriksa apakah Anda memiliki paket-paket yang diperlukan untuk menyelesaikan modul ini dan menginstalnya untuk Anda jika belum tersedia.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Tentukan pertanyaan**

Untuk tujuan kita, kita akan menyatakannya sebagai biner: 'Putih' atau 'Bukan Putih'. Ada juga kategori 'bergaris' dalam dataset kita, tetapi jumlahnya sangat sedikit, jadi kita tidak akan menggunakannya. Kategori ini juga akan hilang setelah kita menghapus nilai null dari dataset.

> 🎃 Fakta menarik, kadang-kadang kita menyebut labu putih sebagai labu 'hantu'. Labu ini tidak mudah diukir, sehingga tidak sepopuler labu oranye, tetapi tampilannya keren! Jadi kita juga bisa merumuskan ulang pertanyaan kita sebagai: 'Hantu' atau 'Bukan Hantu'. 👻

## **Tentang regresi logistik**

Regresi logistik berbeda dari regresi linier, yang telah Anda pelajari sebelumnya, dalam beberapa cara penting.

#### **Klasifikasi biner**

Regresi logistik tidak menawarkan fitur yang sama seperti regresi linier. Regresi logistik memberikan prediksi tentang `kategori biner` ("oranye atau bukan oranye"), sedangkan regresi linier mampu memprediksi `nilai kontinu`, misalnya berdasarkan asal labu dan waktu panen, *berapa harga labu akan naik*.

![Infografis oleh Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Klasifikasi lainnya

Ada jenis regresi logistik lainnya, termasuk multinomial dan ordinal:

- **Multinomial**, yang melibatkan lebih dari satu kategori - "Oranye, Putih, dan Bergaris".

- **Ordinal**, yang melibatkan kategori yang terurut, berguna jika kita ingin mengurutkan hasil secara logis, seperti labu kita yang diurutkan berdasarkan sejumlah ukuran tertentu (mini, sm, med, lg, xl, xxl).

![Regresi multinomial vs ordinal](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Variabel TIDAK harus berkorelasi**

Ingat bagaimana regresi linier bekerja lebih baik dengan variabel yang lebih berkorelasi? Regresi logistik adalah kebalikannya - variabel tidak harus selaras. Ini cocok untuk data ini yang memiliki korelasi yang agak lemah.

#### **Anda membutuhkan banyak data yang bersih**

Regresi logistik akan memberikan hasil yang lebih akurat jika Anda menggunakan lebih banyak data; dataset kecil kita tidak optimal untuk tugas ini, jadi ingatlah hal itu.

✅ Pikirkan jenis data yang cocok untuk regresi logistik

## Latihan - rapikan data

Pertama, bersihkan data sedikit, hapus nilai null, dan pilih hanya beberapa kolom:

1. Tambahkan kode berikut:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Anda selalu dapat melihat sekilas dataframe baru Anda dengan menggunakan fungsi [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) seperti di bawah ini:


In [ ]:
pumpkins_select %>% 
  glimpse()


Mari kita pastikan bahwa kita benar-benar akan mengerjakan masalah klasifikasi biner:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Visualisasi - plot kategorikal
Saat ini Anda telah memuat kembali data labu dan membersihkannya sehingga menghasilkan dataset yang berisi beberapa variabel, termasuk Warna. Mari kita visualisasikan dataframe di notebook menggunakan pustaka ggplot.

Pustaka ggplot menawarkan beberapa cara menarik untuk memvisualisasikan data Anda. Sebagai contoh, Anda dapat membandingkan distribusi data untuk setiap Varietas dan Warna dalam plot kategorikal.

1. Buat plot seperti itu dengan menggunakan fungsi geombar, menggunakan data labu kita, dan menentukan pemetaan warna untuk setiap kategori labu (oranye atau putih):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Dengan mengamati data, Anda dapat melihat bagaimana data Warna berhubungan dengan Variasi.

✅ Berdasarkan plot kategorikal ini, eksplorasi menarik apa yang dapat Anda bayangkan?


### Pra-pemrosesan Data: Pengkodean Fitur

Dataset labu kita berisi nilai string untuk semua kolomnya. Bekerja dengan data kategorikal memang intuitif bagi manusia, tetapi tidak untuk mesin. Algoritma pembelajaran mesin bekerja lebih baik dengan angka. Itulah mengapa pengkodean adalah langkah yang sangat penting dalam fase pra-pemrosesan data, karena memungkinkan kita mengubah data kategorikal menjadi data numerik tanpa kehilangan informasi apa pun. Pengkodean yang baik akan membantu membangun model yang baik.

Untuk pengkodean fitur, terdapat dua jenis pengkode utama:

1. **Ordinal encoder**: Cocok untuk variabel ordinal, yaitu variabel kategorikal di mana datanya memiliki urutan logis, seperti kolom `item_size` dalam dataset kita. Pengkode ini membuat pemetaan sehingga setiap kategori direpresentasikan oleh sebuah angka, yang merupakan urutan kategori dalam kolom tersebut.

2. **Categorical encoder**: Cocok untuk variabel nominal, yaitu variabel kategorikal di mana datanya tidak memiliki urutan logis, seperti semua fitur selain `item_size` dalam dataset kita. Ini menggunakan pengkodean one-hot, yang berarti setiap kategori direpresentasikan oleh kolom biner: variabel yang dikodekan bernilai 1 jika labu tersebut termasuk dalam Varietas tertentu dan 0 jika tidak.

Tidymodels menyediakan satu paket keren lainnya: [recipes](https://recipes.tidymodels.org/) - sebuah paket untuk pra-pemrosesan data. Kita akan mendefinisikan sebuah `recipe` yang menentukan bahwa semua kolom prediktor harus dikodekan menjadi serangkaian bilangan bulat, kemudian `prep` untuk memperkirakan kuantitas dan statistik yang diperlukan oleh operasi apa pun, dan akhirnya `bake` untuk menerapkan perhitungan tersebut pada data baru.

> Biasanya, recipes sering digunakan sebagai pra-pemroses untuk pemodelan, di mana ia mendefinisikan langkah-langkah apa yang harus diterapkan pada sebuah dataset agar siap untuk pemodelan. Dalam kasus tersebut, **sangat disarankan** untuk menggunakan `workflow()` daripada memperkirakan recipe secara manual menggunakan prep dan bake. Kita akan melihat semua ini sebentar lagi.
>
> Namun, untuk saat ini, kita menggunakan recipes + prep + bake untuk menentukan langkah-langkah apa yang harus diterapkan pada sebuah dataset agar siap untuk analisis data, dan kemudian mengekstrak data yang telah diproses dengan langkah-langkah tersebut.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Apa keuntungan menggunakan ordinal encoder untuk kolom Item Size?

### Menganalisis hubungan antar variabel

Setelah kita melakukan pra-pemrosesan data, kita dapat menganalisis hubungan antara fitur-fitur dan label untuk mendapatkan gambaran seberapa baik model dapat memprediksi label berdasarkan fitur-fitur tersebut. Cara terbaik untuk melakukan analisis semacam ini adalah dengan memvisualisasikan data.  
Kita akan kembali menggunakan fungsi ggplot geom_boxplot_ untuk memvisualisasikan hubungan antara Item Size, Variety, dan Color dalam plot kategorikal. Untuk mempermudah visualisasi data, kita akan menggunakan kolom Item Size yang telah dienkode dan kolom Variety yang belum dienkode.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Gunakan plot swarm

Karena Color adalah kategori biner (Putih atau Tidak), maka membutuhkan '[pendekatan khusus](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)' untuk visualisasi.

Cobalah `plot swarm` untuk menunjukkan distribusi warna berdasarkan item_size.

Kita akan menggunakan [paket ggbeeswarm](https://github.com/eclarke/ggbeeswarm) yang menyediakan metode untuk membuat plot gaya beeswarm menggunakan ggplot2. Plot beeswarm adalah cara memplot titik-titik yang biasanya saling tumpang tindih sehingga mereka jatuh bersebelahan satu sama lain.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Sekarang kita memiliki gambaran tentang hubungan antara kategori biner warna dan kelompok ukuran yang lebih besar, mari kita eksplorasi regresi logistik untuk menentukan kemungkinan warna sebuah labu.

## Bangun model Anda

Pilih variabel yang ingin Anda gunakan dalam model klasifikasi dan bagi data menjadi set pelatihan dan pengujian. [rsample](https://rsample.tidymodels.org/), sebuah paket dalam Tidymodels, menyediakan infrastruktur untuk pembagian data dan resampling yang efisien:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Kita sekarang siap untuk melatih model dengan mencocokkan fitur pelatihan ke label pelatihan (warna).

Kita akan mulai dengan membuat resep yang menentukan langkah-langkah praproses yang harus dilakukan pada data kita agar siap untuk pemodelan, yaitu: mengubah variabel kategorikal menjadi serangkaian bilangan bulat. Sama seperti `baked_pumpkins`, kita membuat `pumpkins_recipe` tetapi tidak melakukan `prep` dan `bake` karena ini akan digabungkan ke dalam sebuah alur kerja, yang akan Anda lihat dalam beberapa langkah ke depan.

Ada cukup banyak cara untuk menentukan model regresi logistik di Tidymodels. Lihat `?logistic_reg()` Untuk saat ini, kita akan menentukan model regresi logistik melalui mesin default `stats::glm()`.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Sekarang kita memiliki resep dan spesifikasi model, kita perlu mencari cara untuk menggabungkan keduanya menjadi sebuah objek yang akan terlebih dahulu memproses data (prep+bake di belakang layar), menyesuaikan model pada data yang telah diproses, dan juga memungkinkan aktivitas pasca-pemrosesan jika diperlukan.

Dalam Tidymodels, objek yang praktis ini disebut [`workflow`](https://workflows.tidymodels.org/) dan dengan mudah menyimpan komponen pemodelan Anda.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Setelah alur kerja *ditentukan*, sebuah model dapat `dilatih` menggunakan fungsi [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). Alur kerja akan memperkirakan resep dan memproses data sebelum pelatihan, sehingga kita tidak perlu melakukannya secara manual menggunakan prep dan bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Model mencetak koefisien yang dipelajari selama pelatihan.

Sekarang kita telah melatih model menggunakan data pelatihan, kita dapat membuat prediksi pada data uji menggunakan [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Mari kita mulai dengan menggunakan model untuk memprediksi label untuk set uji kita dan probabilitas untuk setiap label. Ketika probabilitas lebih dari 0.5, kelas prediksi adalah `WHITE`, jika tidak maka `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Sangat bagus! Ini memberikan wawasan lebih tentang cara kerja regresi logistik.

### Pemahaman lebih baik melalui matriks kebingungan

Membandingkan setiap prediksi dengan nilai "ground truth" aktual yang sesuai bukanlah cara yang efisien untuk menentukan seberapa baik model memprediksi. Untungnya, Tidymodels memiliki beberapa trik lagi: [`yardstick`](https://yardstick.tidymodels.org/) - sebuah paket yang digunakan untuk mengukur efektivitas model menggunakan metrik kinerja.

Salah satu metrik kinerja yang terkait dengan masalah klasifikasi adalah [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix). Matriks kebingungan menggambarkan seberapa baik model klasifikasi bekerja. Matriks kebingungan mencatat berapa banyak contoh di setiap kelas yang diklasifikasikan dengan benar oleh model. Dalam kasus kita, matriks ini akan menunjukkan berapa banyak labu oranye yang diklasifikasikan sebagai oranye dan berapa banyak labu putih yang diklasifikasikan sebagai putih; matriks kebingungan juga menunjukkan berapa banyak yang diklasifikasikan ke kategori **yang salah**.

Fungsi [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) dari yardstick menghitung tabulasi silang antara kelas yang diamati dan yang diprediksi.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Mari kita interpretasikan matriks kebingungan. Model kita diminta untuk mengklasifikasikan labu ke dalam dua kategori biner, kategori `putih` dan kategori `tidak-putih`.

-   Jika model Anda memprediksi sebuah labu sebagai putih dan labu tersebut memang termasuk kategori 'putih' dalam kenyataannya, kita menyebutnya sebagai `true positive`, ditunjukkan oleh angka di kiri atas.

-   Jika model Anda memprediksi sebuah labu sebagai tidak putih dan labu tersebut sebenarnya termasuk kategori 'putih' dalam kenyataannya, kita menyebutnya sebagai `false negative`, ditunjukkan oleh angka di kiri bawah.

-   Jika model Anda memprediksi sebuah labu sebagai putih dan labu tersebut sebenarnya termasuk kategori 'tidak-putih' dalam kenyataannya, kita menyebutnya sebagai `false positive`, ditunjukkan oleh angka di kanan atas.

-   Jika model Anda memprediksi sebuah labu sebagai tidak putih dan labu tersebut memang termasuk kategori 'tidak-putih' dalam kenyataannya, kita menyebutnya sebagai `true negative`, ditunjukkan oleh angka di kanan bawah.

| Kebenaran |
|:---------:|


|               |        |       |
|---------------|--------|-------|
| **Prediksi**  | PUTIH  | ORANYE |
| PUTIH         | TP     | FP    |
| ORANYE        | FN     | TN    |

Seperti yang mungkin Anda duga, lebih baik memiliki jumlah true positive dan true negative yang lebih besar serta jumlah false positive dan false negative yang lebih kecil, yang menunjukkan bahwa model bekerja lebih baik.

Matriks kebingungan sangat berguna karena menghasilkan metrik lain yang dapat membantu kita mengevaluasi kinerja model klasifikasi dengan lebih baik. Mari kita bahas beberapa di antaranya:

🎓 Presisi: `TP/(TP + FP)` didefinisikan sebagai proporsi prediksi positif yang benar-benar positif. Juga disebut [nilai prediktif positif](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Recall: `TP/(TP + FN)` didefinisikan sebagai proporsi hasil positif dari jumlah sampel yang benar-benar positif. Juga dikenal sebagai `sensitivitas`.

🎓 Spesifisitas: `TN/(TN + FP)` didefinisikan sebagai proporsi hasil negatif dari jumlah sampel yang benar-benar negatif.

🎓 Akurasi: `TP + TN/(TP + TN + FP + FN)` Persentase label yang diprediksi dengan benar untuk sebuah sampel.

🎓 F Measure: Rata-rata tertimbang dari presisi dan recall, dengan nilai terbaik adalah 1 dan terburuk adalah 0.

Mari kita hitung metrik-metrik ini!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Visualisasikan kurva ROC dari model ini

Mari kita lakukan satu visualisasi lagi untuk melihat apa yang disebut [`kurva ROC`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


Kurva ROC sering digunakan untuk melihat output dari sebuah pengklasifikasi dalam hal positif benar vs. positif salah. Kurva ROC biasanya menampilkan `True Positive Rate`/Sensitivitas pada sumbu Y, dan `False Positive Rate`/1-Spesifisitas pada sumbu X. Oleh karena itu, kemiringan kurva dan ruang antara garis tengah dan kurva menjadi penting: Anda menginginkan kurva yang dengan cepat naik dan melewati garis tersebut. Dalam kasus kita, ada positif salah di awal, lalu garisnya naik dan melewati dengan benar.

Terakhir, mari gunakan `yardstick::roc_auc()` untuk menghitung Area Under the Curve yang sebenarnya. Salah satu cara untuk menginterpretasikan AUC adalah sebagai probabilitas bahwa model memberi peringkat lebih tinggi pada contoh positif acak dibandingkan dengan contoh negatif acak.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Hasilnya sekitar `0.975`. Mengingat bahwa AUC berkisar dari 0 hingga 1, Anda menginginkan skor yang besar, karena model yang 100% benar dalam prediksinya akan memiliki AUC sebesar 1; dalam hal ini, modelnya *cukup baik*.

Dalam pelajaran mendatang tentang klasifikasi, Anda akan belajar cara meningkatkan skor model Anda (seperti menangani data yang tidak seimbang dalam kasus ini).

## 🚀Tantangan

Masih banyak yang bisa dipelajari tentang regresi logistik! Tetapi cara terbaik untuk belajar adalah dengan bereksperimen. Temukan dataset yang cocok untuk jenis analisis ini dan bangun model dengannya. Apa yang Anda pelajari? tip: coba [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) untuk dataset yang menarik.

## Tinjauan & Studi Mandiri

Baca beberapa halaman pertama dari [makalah ini dari Stanford](https://web.stanford.edu/~jurafsky/slp3/5.pdf) tentang beberapa penggunaan praktis regresi logistik. Pikirkan tentang tugas-tugas yang lebih cocok untuk salah satu jenis regresi yang telah kita pelajari sejauh ini. Apa yang akan bekerja paling baik?



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
